In [1]:
import pandas as pd
from lxml import etree
import os
from bs4 import BeautifulSoup
import requests
import xml.etree.ElementTree as ET
import validators
import datetime
import re

In [2]:
#parameters
characters_for_name=60
legislature='dar/01'
cycle='14'

In [3]:
#utils

def add_zeros(int_):
    if len(str(int_))<2:
        return '00'+str(int_)
    if len(str(int_))<3:
        return '0'+str(int_)
    if len(str(int_))==3:
        return str(int_)

def get_text_from_html(url__):
    from bs4 import BeautifulSoup
    res = requests.get(url__)
    html_page = res.content
    soup = BeautifulSoup(html_page, 'html.parser')
    return str(soup.noscript)

In [4]:
#regex
def check_dialog(text_):
    pat="(?:</p><p>.*?\(.*?\): —)"
    match=re.findall(pat, text_)
    if match is None :
        return 'no_dialog'
    else:
        for m in match:
            
            return m

def extract_dialog(full_text,characters_for_name):    
    if full_text is None:
        return None
    else:
        dialogs=[(m.start(0), m.end(0)) for m in re.finditer('</p><p></p><p>(.+?): —', full_text)]
        out=[]
        if len(dialogs)==1:
            out.append(full_text[dialogs[0][1]-characters_for_name:])
        if len(dialogs)>1:
            for i in range(0, len(dialogs)-1):
                out.append(full_text[dialogs[i][1]-characters_for_name:dialogs[i+1][0]])
            out.append(full_text[dialogs[len(dialogs)-1][1]-50:])
        return out

    
def extract_party_name(dialogs):
    if dialogs is None:
        return None
    else:
        res=[]
        for i in dialogs:
                    positions=[(m.start(0), m.end(0)) for m in re.finditer('\((.+?)\): —', i)]
                    if len(positions)==0:
                        res.append(['No','No'])
                    else:
                        party=re.findall('\((.+?)\)',i)
                        name_aux=i[positions[0][1]-50:positions[0][1]]
                        name=re.sub(r'\b\w{1,2}\b', '', name_aux).replace('.','').replace ('()','').replace('  ','').replace('<','').replace('>','').replace('/','').replace(': —','')
                        if len(party)>0:
                            party=party[0]
                        res.append([party,name])
        return res

def add_speech_next_page(df):
    for pi in range(1,df.page.max()+1):
        if len(df[df.page==pi].speech.values[0])>0:
            speeches=df[df.page==pi].speech.values[0]
            if '</noscript>' in speeches[-1]:
                
                for n in range(1,df.page.max()-pi):
                    if ': —' in df[df.page==pi+n].text_1.values[0]:
                        
                        in_=df[df.page==pi+n].text_1.values[0].find(': —')
                        df[df.page==pi].speech.values[0][-1]=df[df.page==pi].speech.values[0][-1]+' '+str(df[df.page==pi+n].text_1.values[0][0:in_])
                        
                        break
                    else:
                        
                        df[df.page==pi].speech.values[0][-1]=df[df.page==pi].speech.values[0][-1]+' '+str(df[df.page==pi+n].text_1.values[0])
    return df

In [5]:
current_directory="C:\\Users\caperei"
cycle='14'
session='1'
data_directory=f"\portuguese_open_data\data\cycle\{cycle}\session\{session}\\numbers_dates_pages.csv"
input_data=pd.read_csv(r"C:\Users\caperei\portuguese_open_data\data\cyle\14\session\1\numbers_dates_pages.csv")

In [6]:
input_data.head(3)

,number,date,pages
0,1,25/10/2019,13
1,2,30/10/2019,124
2,3,31/10/2019,26


In [7]:
df = pd.DataFrame(columns=['legislature','cycle','session','number','date','page', 'text_1','url'])        
session=['01'] #['01','02','03']
numbers_dates=list(zip(input_data.number, input_data.date,input_data.pages ))[0:1]
pages=input_data.pages.values[0:1]

In [8]:
s_c=[]
n_c=[]
d_c=[]
p_c=[]
u_c=[]
t_c=[]
for s in session:
    for number, date, page in numbers_dates:
        number=add_zeros(number)
        date=datetime.datetime.strptime(date, '%d/%m/%Y').strftime('%Y-%m-%d')
        for page_i in range(1,page+1):
                url_=f'https://debates.parlamento.pt/catalogo/r3/{legislature}/{cycle}/{s}/{number}/{date}/{page_i}'
                if validators.url(url_):
                    #print (url_)
                    u_c.append(url_)
                    p_c.append(page_i)
                    t_c.append(get_text_from_html(url_))
                    s_c.append(s)
                    n_c.append(number)
                    d_c.append(date)
df['page']=p_c
df['session']=s_c
df['number']=n_c
df['date']=d_c

df['text_1']=t_c
df['url']=u_c
df['legislature']=legislature
df['cycle']=cycle    

In [9]:
#df.to_pickle('portuguese_transcripts.pkl') 
#df = pd.read_pickle('portuguese_transcripts.pkl')

In [10]:
#df['dialogs']=[check_dialog(t) for t in df.text_1]

In [11]:
#add columns
df['speech']=[extract_dialog(t,characters_for_name) for t in df.text_1 ]
df['party']=[extract_party_name(t) for t in df.speech ]
df=add_speech_next_page(df)

In [12]:
#re arrange by speaker and party
df1=df.explode(['speech', 'party']).reset_index(drop=True)
df2=df1.dropna(subset=['party'])
df2['party_s']=[t[0] for t in df2.party]
df2['speaker']=[t[1] for t in df2.party]
df2=df2[df2.party_s!='No']

<ipython-input-12-1f947bc0064b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['party_s']=[t[0] for t in df2.party]
<ipython-input-12-1f947bc0064b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['speaker']=[t[1] for t in df2.party]


In [13]:
df2

,legislature,cycle,session,number,date,page,text_1,url,speech,party,party_s,speaker
1,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,es.</p><p></p><p>A Sr.ª Ana Catarina Mendonça ...,"[PS, Ana Catarina Mendonça Mendes ]",PS,Ana Catarina Mendonça Mendes
3,dar/01,14,01,001,2019-10-25,2,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,</p><p></p><p>O Sr. Secretário (Duarte Pacheco...,"[Duarte Pacheco, Secretário (Duarte Pacheco)]",Duarte Pacheco,Secretário (Duarte Pacheco)
5,dar/01,14,01,001,2019-10-25,3,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>3 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,o. </p><p></p><p>O Sr. Pedro Delgado Alves (PS...,"[PS, Pedro Delgado Alves ]",PS,Pedro Delgado Alves
8,dar/01,14,01,001,2019-10-25,6,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,cretário. </p><p></p><p>O Sr. Secretário (Duar...,"[Duarte Pacheco, Secretário (Duarte Pacheco)]",Duarte Pacheco,Secretário (Duarte Pacheco)
9,dar/01,14,01,001,2019-10-25,6,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,p><p>O Sr. Presidente (Eduardo Ferro Rodrigues...,"[Eduardo Ferro Rodrigues, Presidente (Eduardo ...",Eduardo Ferro Rodrigues,Presidente (Eduardo Ferro Rodrigues)
12,dar/01,14,01,001,2019-10-25,8,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,</p><p>A Sr.ª Ana Catarina Mendonça Mendes (PS...,"[PS, Ana Catarina Mendonça Mendes ]",PS,Ana Catarina Mendonça Mendes
14,dar/01,14,01,001,2019-10-25,9,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>9 ...,https://debates.parlamento.pt/catalogo/r3/dar/...,grão. </p><p></p><p>O Sr. Fernando Negrão (PSD...,"[PSD, grão Fernando Negrão (PSD)]",PSD,grão Fernando Negrão (PSD)
16,dar/01,14,01,001,2019-10-25,10,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,lipe Soares. </p><p></p><p>O Sr. Pedro Filipe ...,"[BE, Pedro Filipe Soares ]",BE,Pedro Filipe Soares
18,dar/01,14,01,001,2019-10-25,10,<noscript>\n<p>I SÉRIE — NÚMERO 1 </p><p></p><...,https://debates.parlamento.pt/catalogo/r3/dar/...,iveira. </p><p></p><p>O Sr. João Oliveira (PCP...,"[PCP, iveira João Oliveira (PCP)]",PCP,iveira João Oliveira (PCP)
20,dar/01,14,01,001,2019-10-25,11,<noscript>\n<p>26 DE OUTUBRO DE 2019 </p><p>11...,https://debates.parlamento.pt/catalogo/r3/dar/...,>Meireles. </p><p></p><p>A Sr.ª Cecília Meirel...,"[CDS-PP, Cecília Meireles (CDS-)]",CDS-PP,Cecília Meireles (CDS-)
